In [2]:
from dev_spark_session import DevSparkSession

# Initialize Spark with Hive support (so CREATE/DROP TABLE work)
spark = DevSparkSession().spark

# Your Delta folder path
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"
dimSymbol_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/dim-symbol"
dimDate_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/dim-date"
fact_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/fact-daily-summary"

# ## re-create table 
# truncate_schema = spark.read.format("delta").load(silver_path).schema
# truncate_silver = spark.createDataFrame([], schema= truncate_schema)
# truncate_silver.write.format("delta").mode("overwrite").save(silver_path)

ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d8f4ff66-0201-4168-b560-cbba3d143310;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 87ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

### Silver

In [ ]:
spark.sql(f"""
CREATE TABLE delta_table (
    Symbol                   STRING,
    ExchangeName             STRING,
    Currency                 STRING,
    Type                     STRING,
    ExchangeTimeZone         STRING,
    Volume                   INTEGER,
    High                     DECIMAL,
    Low                      DECIMAL,
    Close                    DECIMAL,
    Open                     DECIMAL,
    Date                     DATE,
    __CurrentFlag            BOOLEAN,
    __DeletedFlag            BOOLEAN,
    __EffectiveStartDateTime TIMESTAMP,
    __EffectiveEndDateTime   TIMESTAMP,
    __lastmodified           TIMESTAMP,
    __HashKey                STRING,
    __HashValue              STRING,
    __CreatedBatchLogId      STRING,
    __UpdatedBatchLogId      STRING,
    __CreateDateTime         TIMESTAMP,
    __UpdateDateTime         TIMESTAMP
)
USING DELTA
LOCATION '{silver_path}'
""")

# 3) Verify registration and schema
spark.sql("SHOW TABLES").show(truncate=False)
spark.sql("DESCRIBE delta_table").show(truncate=False)


### Dim Date

In [ ]:

# 2) Create the table with full schema, pointing at that folder
spark.sql(f"""
CREATE TABLE dim_date (
    
    DateID              INTEGER,
    Date                DATE,
    Day                 INTEGER,
    DayOfWeek           VARCHAR(20),
    DayOfWeekNumber     INTEGER,
    MonthName           VARCHAR(20),
    MonthNumber         INTEGER,
    Year                INTEGER,
    YearMonth           INTEGER
   
)
USING DELTA
LOCATION '{dimDate_path}'
""")

# 3) Verify registration and schema
spark.sql("SHOW TABLES").show(truncate=False)
spark.sql("DESCRIBE delta_table").show(truncate=False)


25/04/21 23:39:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|default  |dim_date |false      |
+---------+---------+-----------+



AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `delta_table` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 9;
'DescribeRelation false, [col_name#561, data_type#562, comment#563]
+- 'UnresolvedTableOrView [delta_table], DESCRIBE TABLE, true


25/04/22 03:15:11 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1043846 ms exceeds timeout 120000 ms
25/04/22 03:15:11 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/22 03:23:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

### Dim Symbol

In [ ]:

# 2) Create the table with full schema, pointing at that folder
spark.sql(f"""
CREATE TABLE dimsymbol_table (
    SymbolSID                BIGINT,
    Symbol                   STRING,
    ExchangeName             STRING,
    Currency                 STRING,
    Type                     STRING,
    ExchangeTimeZone         STRING,
    __CurrentFlag            BOOLEAN,
    __DeletedFlag            BOOLEAN,
    __EffectiveStartDateTime TIMESTAMP,
    __EffectiveEndDateTime   TIMESTAMP,
    __BusinessKeyHash        VARCHAR(64),
    __Hash1Type              VARCHAR(64),
    __Hash2Type              VARCHAR(64),
    __CreatedBatchLogId      VARCHAR(50),
    __UpdatedBatchLogId      VARCHAR(50),
    __CreateDateTime         TIMESTAMP,
    __UpdateDateTime         TIMESTAMP
)
USING DELTA
LOCATION '{dimSymbol_path}'
""")

# # 3) Verify registration and schema
# spark.sql("SHOW TABLES").show(truncate=False)
# spark.sql("DESCRIBE dimsymbol_table").show(truncate=False)


### Fact Daily Summary

In [ ]:

# 2) Create the table with full schema, pointing at that folder
spark.sql(f"""
CREATE TABLE fact_trading (
    
    TransactionSID           BIGINT,
    SymbolSID                BIGINT,
    Volume                   INTEGER,
    High                     DECIMAL,
    Low                      DECIMAL,
    Close                    DECIMAL,
    Open                     DECIMAL,
    DateID                   INTEGER,
    __DeletedFlag            BOOLEAN,
    __FactKeyHash            VARCHAR(64),
    __CreatedBatchLogId      VARCHAR(50),
    __CreateDateTime         TIMESTAMP
)
USING DELTA
LOCATION '{fact_path}'
""")

# 3) Verify registration and schema
spark.sql("SHOW TABLES").show(truncate=False)
spark.sql("DESCRIBE delta_table").show(truncate=False)
